In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1230819,2021-04-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1230820,2021-05-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1230821,2021-05-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1230822,2021-05-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32867,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32869,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32871,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32873,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32875,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
757345,2021-04-30,Arkansas,Arkansas,5001,2094,36.00,5000,Arkansas,AR,Arkansas,State,3017804
757347,2021-05-01,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
757349,2021-05-02,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804
757351,2021-05-03,Arkansas,Arkansas,5001,2095,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1230819,2021-04-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1230820,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1230821,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1230822,2021-05-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-04') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
469,2021-05-04,Snohomish,Washington,36446,569.00,53061,WA,County,822083,4433.37,69.21
936,2021-05-04,Cook,Illinois,538095,10551.00,17031,IL,County,5150233,10447.97,204.86
1402,2021-05-04,Orange,California,270444,4978.00,6059,CA,County,3175692,8516.07,156.75
1867,2021-05-04,Maricopa,Arizona,539080,9886.00,4013,AZ,County,4485414,12018.51,220.40
2332,2021-05-04,Los Angeles,California,1234865,23942.00,6037,CA,County,10039107,12300.55,238.49
...,...,...,...,...,...,...,...,...,...,...,...
1230131,2021-05-04,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1230335,2021-05-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1230508,2021-05-04,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1230677,2021-05-04,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
469,2021-05-04,Snohomish,Washington,36446,569.00,53061,WA,County,822083,4433.37,69.21,69.21,4433.37
936,2021-05-04,Cook,Illinois,538095,10551.00,17031,IL,County,5150233,10447.97,204.86,204.86,10447.97
1402,2021-05-04,Orange,California,270444,4978.00,6059,CA,County,3175692,8516.07,156.75,156.75,8516.07
1867,2021-05-04,Maricopa,Arizona,539080,9886.00,4013,AZ,County,4485414,12018.51,220.40,220.40,12018.51
2332,2021-05-04,Los Angeles,California,1234865,23942.00,6037,CA,County,10039107,12300.55,238.49,238.49,12300.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230131,2021-05-04,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1230335,2021-05-04,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1230508,2021-05-04,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1230677,2021-05-04,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
791922,2021-05-04,Hale,Alabama,2217,76.00,1065,AL,County,14651,15132.07,518.74,518.74,15132.07,1
862017,2021-05-04,Clarke,Alabama,3503,60.00,1025,AL,County,23622,14829.40,254.00,254.00,14829.40,2
619581,2021-05-04,Lowndes,Alabama,1384,53.00,1085,AL,County,9726,14229.90,544.93,544.93,14229.90,3
460956,2021-05-04,Franklin,Alabama,4238,82.00,1059,AL,County,31362,13513.17,261.46,261.46,13513.17,4
549349,2021-05-04,Etowah,Alabama,13796,352.00,1055,AL,County,102268,13490.05,344.19,344.19,13490.05,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166640,2021-05-04,Platte,Wyoming,623,11.00,56031,WY,County,8393,7422.85,131.06,131.06,7422.85,19
755744,2021-05-04,Converse,Wyoming,997,17.00,56009,WY,County,13822,7213.14,122.99,122.99,7213.14,20
939629,2021-05-04,Lincoln,Wyoming,1400,12.00,56023,WY,County,19830,7060.01,60.51,60.51,7060.01,21
1045825,2021-05-04,Niobrara,Wyoming,157,2.00,56027,WY,County,2356,6663.84,84.89,84.89,6663.84,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
619581,2021-05-04,Lowndes,Alabama,1384,53.00,1085,AL,County,9726,14229.90,544.93,544.93,14229.90,3,1
791922,2021-05-04,Hale,Alabama,2217,76.00,1065,AL,County,14651,15132.07,518.74,518.74,15132.07,1,2
263466,2021-05-04,Walker,Alabama,7205,279.00,1127,AL,County,63521,11342.71,439.22,439.22,11342.71,28,3
547725,2021-05-04,Clay,Alabama,1533,56.00,1027,AL,County,13235,11582.92,423.12,423.12,11582.92,21,4
668178,2021-05-04,Greene,Alabama,922,34.00,1063,AL,County,8111,11367.28,419.18,419.18,11367.28,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666561,2021-05-04,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
916206,2021-05-04,Uinta,Wyoming,2204,13.00,56041,WY,County,20226,10896.87,64.27,64.27,10896.87,5,20
939629,2021-05-04,Lincoln,Wyoming,1400,12.00,56023,WY,County,19830,7060.01,60.51,60.51,7060.01,21,21
262230,2021-05-04,Teton,Wyoming,3741,9.00,56039,WY,County,23464,15943.57,38.36,38.36,15943.57,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4433.37,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4433.37,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4433.37,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4433.37,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.21,4433.37,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227935,2021-04-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1227936,2021-05-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1227937,2021-05-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1227938,2021-05-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
788638,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,1.00
788639,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
788640,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
788641,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,518.74,15132.07,2,1,0.00
788642,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,518.74,15132.07,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955353,2021-04-30,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
955354,2021-05-01,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
955355,2021-05-02,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00
955356,2021-05-03,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
617099,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,1.00,0.00
617100,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
617101,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
617102,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
617103,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14229.90,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615069,2021-04-30,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,30.86,10985.08,23,4,17.00,0.00
615070,2021-05-01,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,30.86,10985.08,23,4,0.00,0.00
615071,2021-05-02,Albany,Wyoming,4242,11.00,56001,WY,County,38880,10910.49,28.29,30.86,10985.08,23,4,0.00,0.00
615072,2021-05-03,Albany,Wyoming,4266,11.00,56001,WY,County,38880,10972.22,28.29,30.86,10985.08,23,4,24.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-04') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
314522,2021-05-04,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,5.00,0.00
2332,2021-05-04,Los Angeles,California,1234865,23942.00,6037,CA,County,10039107,12300.55,238.49,238.49,12300.55,2,5,217.00,-1.00
156596,2021-05-04,San Bernardino,California,296188,4659.00,6071,CA,County,2180085,13586.08,213.71,213.71,13586.08,3,4,42.00,4.00
550925,2021-05-04,Inyo,California,1422,38.00,6027,CA,County,18039,7882.92,210.65,210.65,7882.92,4,24,0.00,0.00
70017,2021-05-04,Stanislaus,California,61606,1047.00,6099,CA,County,550660,11187.67,190.14,190.14,11187.67,5,9,60.00,1.00
32442,2021-05-04,Riverside,California,298777,4574.00,6065,CA,County,2470546,12093.56,185.14,185.14,12093.56,6,6,205.00,0.00
55705,2021-05-04,San Joaquin,California,72724,1382.00,6077,CA,County,762148,9541.98,181.33,181.33,9541.98,7,19,52.00,17.00
86814,2021-05-04,Tulare,California,49496,837.00,6107,CA,County,466195,10617.02,179.54,179.54,10617.02,8,10,19.00,0.00
31594,2021-05-04,Fresno,California,101405,1678.00,6019,CA,County,999101,10149.62,167.95,167.95,10149.62,9,14,33.00,0.00
499253,2021-05-04,Merced,California,31682,454.00,6047,CA,County,277680,11409.54,163.50,163.50,11409.54,10,8,25.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1166191,2021-05-04,Lassen,California,5692,24.00,6035,CA,County,30573,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
314522,2021-05-04,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,5.00,0.00
668120,2021-05-04,Kings,California,22911,246.00,6031,CA,County,152940,14980.38,160.85,160.85,14980.38,11,3,1.00,0.00
156596,2021-05-04,San Bernardino,California,296188,4659.00,6071,CA,County,2180085,13586.08,213.71,213.71,13586.08,3,4,42.00,4.00
2332,2021-05-04,Los Angeles,California,1234865,23942.00,6037,CA,County,10039107,12300.55,238.49,238.49,12300.55,2,5,217.00,-1.00
32442,2021-05-04,Riverside,California,298777,4574.00,6065,CA,County,2470546,12093.56,185.14,185.14,12093.56,6,6,205.00,0.00
196893,2021-05-04,Kern,California,108719,1358.00,6029,CA,County,900202,12077.18,150.86,150.86,12077.18,14,7,67.00,4.00
499253,2021-05-04,Merced,California,31682,454.00,6047,CA,County,277680,11409.54,163.50,163.50,11409.54,10,8,25.00,0.00
70017,2021-05-04,Stanislaus,California,61606,1047.00,6099,CA,County,550660,11187.67,190.14,190.14,11187.67,5,9,60.00,1.00
86814,2021-05-04,Tulare,California,49496,837.00,6107,CA,County,466195,10617.02,179.54,179.54,10617.02,8,10,19.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
314522,2021-05-04,Imperial,California,28280,721.00,6025,CA,County,181215,15605.77,397.87,397.87,15605.77,1,2,5.00,0.00
2332,2021-05-04,Los Angeles,California,1234865,23942.00,6037,CA,County,10039107,12300.55,238.49,238.49,12300.55,2,5,217.00,-1.00
156596,2021-05-04,San Bernardino,California,296188,4659.00,6071,CA,County,2180085,13586.08,213.71,213.71,13586.08,3,4,42.00,4.00
550925,2021-05-04,Inyo,California,1422,38.00,6027,CA,County,18039,7882.92,210.65,210.65,7882.92,4,24,0.00,0.00
70017,2021-05-04,Stanislaus,California,61606,1047.00,6099,CA,County,550660,11187.67,190.14,190.14,11187.67,5,9,60.00,1.00
32442,2021-05-04,Riverside,California,298777,4574.00,6065,CA,County,2470546,12093.56,185.14,185.14,12093.56,6,6,205.00,0.00
55705,2021-05-04,San Joaquin,California,72724,1382.00,6077,CA,County,762148,9541.98,181.33,181.33,9541.98,7,19,52.00,17.00
86814,2021-05-04,Tulare,California,49496,837.00,6107,CA,County,466195,10617.02,179.54,179.54,10617.02,8,10,19.00,0.00
31594,2021-05-04,Fresno,California,101405,1678.00,6019,CA,County,999101,10149.62,167.95,167.95,10149.62,9,14,33.00,0.00
499253,2021-05-04,Merced,California,31682,454.00,6047,CA,County,277680,11409.54,163.50,163.50,11409.54,10,8,25.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15605.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4122,04/30/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
4123,05/01/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
4124,05/02/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00
4125,05/03/21,Lassen,5692,24.00,18617.73,78.50,78.50,18617.73,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,397.87,15605.77,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,397.87,15605.77,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4208,04/30/21,Merced,31594,454.00,11377.85,163.50,163.50,11409.54,10,8,37.00,0.00
4209,05/01/21,Merced,31594,454.00,11377.85,163.50,163.50,11409.54,10,8,0.00,0.00
4210,05/02/21,Merced,31594,454.00,11377.85,163.50,163.50,11409.54,10,8,0.00,0.00
4211,05/03/21,Merced,31657,454.00,11400.53,163.50,163.50,11409.54,10,8,63.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)